## Deliverable 2. Create a Customer Travel Destinations Map.

In [32]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [33]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hithadhoo,MV,-0.6000,73.0833,85.32,73,100,16.33,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,76.37,71,12,5.64,few clouds
2,2,Mataura,NZ,-46.1927,168.8643,57.47,87,12,3.36,few clouds
3,3,Busselton,AU,-33.6500,115.3333,61.05,82,67,1.61,broken clouds
4,4,Saint Anthony,US,45.0205,-93.2180,46.00,87,100,17.27,overcast clouds


In [34]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would prefer for your trip?"))
max_temp=float(input("What is the maximum temperature you would prefer for your trip?"))


What is the minimum temperature you would prefer for your trip?90
What is the maximum temperature you would prefer for your trip?120


In [35]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & \
                                     (city_data_df["Max Temp"]>=min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
168,168,Jalu,LY,29.0331,21.5482,90.81,18,99,23.17,overcast clouds
228,228,Cadereyta,MX,25.5833,-99.9833,90.59,34,74,21.23,broken clouds
244,244,Bilma,NE,18.6853,12.9164,92.21,11,49,7.92,scattered clouds
247,247,Oro Valley,US,32.3909,-110.9665,94.08,6,0,11.01,clear sky
394,394,Diapaga,BF,12.0708,1.7889,90.55,28,29,2.64,scattered clouds


In [36]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                11
City                   11
Country                11
Lat                    11
Lng                    11
Max Temp               11
Humidity               11
Cloudiness             11
Wind Speed             11
Current Description    11
dtype: int64

In [37]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [38]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
168,Jalu,LY,90.81,overcast clouds,29.0331,21.5482,
228,Cadereyta,MX,90.59,broken clouds,25.5833,-99.9833,
244,Bilma,NE,92.21,scattered clouds,18.6853,12.9164,
247,Oro Valley,US,94.08,clear sky,32.3909,-110.9665,
394,Diapaga,BF,90.55,scattered clouds,12.0708,1.7889,
542,Safford,US,92.32,clear sky,32.8340,-109.7076,
637,Aguie,NE,90.90,scattered clouds,13.5060,7.7786,
639,San Antonio,US,91.15,scattered clouds,29.4241,-98.4936,
667,Matamoros,MX,96.33,overcast clouds,25.5333,-103.2500,
670,Deshnok,IN,90.01,scattered clouds,27.7986,73.3447,


In [39]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"]=hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found. Skipping")
        

Hotel not found. Skipping
Hotel not found. Skipping
Hotel not found. Skipping


In [40]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [41]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [42]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} Fahrenheit</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))